## Fazendo Importações e passando os parâmetros necessários para realizar as resquisições

In [27]:
import hashlib
import time
import requests
from google.colab import userdata
import sqlite3
import pandas as pd

public_key = userdata.get('public_key')
private_key = userdata.get('private_key')

ts = str(time.time())
to_hash = ts + private_key + public_key
hash_digest = hashlib.md5(to_hash.encode()).hexdigest()
auth_params = {
        "ts": ts,
        "apikey": public_key,
        "hash": hash_digest,
        "limit": 100
    }

In [ ]:
url1 = "https://gateway.marvel.com/v1/public/characters"
url2 = "https://gateway.marvel.com/v1/public/comics"

## Função que consome o endpoint da API

In [54]:
def get_data(url):
  total_data = 0
  offset = 0
  data = pd.DataFrame()

  # Garante a entrada no loop na primeira vez
  while offset < total_data or total_data == 0:
      print(offset)
      auth_params['offset'] = offset
      response = requests.get(url, auth_params)
      response = response.json()

      # Atualiza total_data após a primeira requisição
      if total_data == 0:
          total_data = response['data']['total']

      # Verifica se a resposta contém dados
      if not response['data']['results']:
          break

      response = pd.json_normalize(response['data']['results'])
      df1 = response
      data = pd.concat([data, df1])
      offset += len(df1)

  return data

In [56]:
characters = get_data(url1)

conn = sqlite3.connect('banco.db')
columns = ['id', 'name','description']

characters[columns].to_sql('characters', conn, if_exists='replace', index=False)

conn.close()